In [2]:
import sys
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.cross_decomposition import CCA
import numpy as np
import pandas as pd
import json
import spacy
nlp = spacy.load('en')

In [3]:
# Read in the list of qustions from the file
qs_file = "File with list of default questions"
base_qs = pd.read_csv(qs_file)

In [4]:
def get_app_qs(filename):
    # Get file with curated questions
    # This is specific to the type of file being used
    # i.e. can add code here are needed
    # Returns a dataframe

    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

In [5]:
def get_cca(veca, vecb):
    """ Get CCA and update cos scores """
    # Get the first canonical correlation
    cca_vals = []
    for qb in vecb:
        cca = CCA(n_components=1)
        U = np.array(veca).reshape(-1, 1)
        V = np.array(qb).reshape(-1, 1)
        U_c, V_c = cca.fit_transform(U, V)
        cca_score = np.corrcoef(U_c.T, V_c.T)[0, 1]
        cca_vals.append(np.square(cca_score))
    return(cca_vals)

In [6]:
"""
Remove stopword and other POS words which may not be relevant to out search
"""

def remove_stop_words(text):
    doc = nlp(text.lower())
    ct = []
    for token in doc:
        if not (token.is_stop) and not (token.dep_ == 'punct'):
            ct.append(token.text)
    ct.append('XXX') if len(ct) < 1 else None
    return(' '.join(word for word in ct))

In [7]:
df1 = get_app_qs("CURATED FILE")


In [7]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" 
embed = hub.Module(module_url)

sts_input1 = tf.placeholder(tf.string, shape=(None))
sts_input2 = tf.placeholder(tf.string, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1), axis=1)
sts_encode2 = tf.nn.l2_normalize(embed(sts_input2), axis=1)

cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
clip_cosine_similarities = tf.clip_by_value(cosine_similarities, 0.0, 1.0)
sim_scores = 1.0 - tf.divide(tf.acos(clip_cosine_similarities), 3.14)

def get_scores(session, text_a, text_b):
    """Returns the similarity scores"""
    emba, embb, scores= session.run(
        [sts_encode1, sts_encode2, sim_scores],
        feed_dict={
            sts_input1: text_a,
            sts_input2: text_b
        })
    return(emba, embb, scores)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_11
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_12:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/v

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv2/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Ini

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/conv2/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Ini

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/kernel:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/conv2/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/tmp/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_5/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Ini

In [8]:
# Use a TF placeholder for the average
avg_input1 = tf.placeholder(tf.float64, shape=(None))
avg_input2 = tf.placeholder(tf.float64, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
avg_encode1 = tf.nn.l2_normalize(avg_input1, axis=1)
avg_encode2 = tf.nn.l2_normalize(avg_input2, axis=1)

# Get cosine similarity for comparison
avg_cosine_similarities = tf.reduce_sum(tf.multiply(avg_encode1, avg_encode2), axis=1)
avg_clip_cosine_similarities = tf.clip_by_value(avg_cosine_similarities, 0.0, 1.0)
avg_sim_scores = 1.0 - tf.acos(avg_clip_cosine_similarities)

def get_mean_score(session, query, examples):
    labels = examples.groupby('label')['text'].count().sort_values(ascending=False)
    """Get the averages vector values for a group"""
    data=[]
    for name, group in examples.groupby('label'):
        zero_vec = np.array([0] * 512)
        zero_vec.astype('float64')
        grp_size = len(group['embb'].tolist())
        for l in group['embb'].tolist():
            emb = (np.array(l))
            zero_vec = np.add(zero_vec, emb, casting="unsafe")
        mean_vec = np.divide(zero_vec, grp_size)
        # Now need to create a DF of [label, question]
        data.append([name, mean_vec])
    avg_df = pd.DataFrame(data, columns=['label', 'average'])
    # Add new question column to DF
    avg_df["question"] = examples['emba']

    """Returns the similarity scores for the group averages"""
    emba, avg_embb, avg_scores = session.run(
        [avg_encode1, avg_encode2, avg_sim_scores],
        feed_dict={
            avg_input1: avg_df['question'].tolist(),
            avg_input2: avg_df['average'].tolist()
        })
    return(avg_df, emba, avg_embb, avg_scores)

In [9]:
"""
Find the best match using the sentence encodings
"""

results = []
    
long_queries = [q for q in base_qs['Question']]
long_examples = [e for e in df1['text']]
queries = [remove_stop_words(q) for q in base_qs['Question']]
examples = [remove_stop_words(e) for e in df1['text']]

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    for i, (q, l_q) in enumerate(zip(queries, long_queries)):
        emba, embb, scores = get_scores(session, [q], examples)
        cca_scores = get_cca(emba, embb)
        df1['score'] = cca_scores
        sort_df = df1.sort_values('score', ascending=False)
        best = sort_df.head(n=1)
        ans = (best['text']).tolist()[0]
        results.append([q, l_q, (best['label']).tolist()[0], 
                        ans, remove_stop_words(ans),
                        np.round((best['score']).tolist()[0], 3)])
        if i % 10 == 0:
            print(f"Processed {i} queries")
    df = pd.DataFrame(results, columns=['Query', 'Long Query', 'Label', 'Long Question', 'Question', 'Score'])

Processed 0 queries
Processed 10 queries
Processed 20 queries
Processed 30 queries


In [ ]:
"""
Find the best match using the average of the sentence encodings for the curated examples
"""

results = []
    
queries = [q for q in base_qs['Question']]
examples = [e for e in df1['text']]
    
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    for i, q in enumerate(queries):
        emba, embb, scores = get_scores(session, [q], examples)
        # Store the embeddings in the DF
        df1['emba'] = np.array(emba).tolist()*len(scores)
        df1['embb'] = np.array(embb).tolist()
        avg, emba, embb, avg_scores = get_mean_score(session, [q], df1)
        avg['score'] = avg_scores
        sort_df = avg.sort_values('score', ascending=False)
        best = sort_df.head(n=1)
        
        results.append([q, (best['label']).tolist()[0], 
                        np.round((best['score']).tolist()[0], 3)])
        if i % 10 == 0:
            print(f"Processed {i} queries")
    df = pd.DataFrame(results, columns=['Query', 'Label', 'Score'])

In [10]:
df.to_csv('use_input_rec1.csv')

In [ ]:
abs = (pd.read_csv('use_test_rec1.csv')).sort_values('Query', ascending=False)
avg = (pd.read_csv('use_avg_rec1.csv')).sort_values('Query', ascending=False)